![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [89]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below
import xgboost as xgb

# Exploratory Data Analysis

## Read the .csv file

In [90]:
rental_info = pd.read_csv('rental_info.csv',
                         parse_dates=['rental_date', 'return_date'])
rental_info.head()

,rental_date,return_date,amount,release_year,rental_rate,length,replacement_cost,special_features,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
0,2005-05-25 02:54:33+00:00,2005-05-28 23:40:33+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
1,2005-06-15 23:19:16+00:00,2005-06-18 19:24:16+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
2,2005-07-10 04:27:45+00:00,2005-07-17 10:11:45+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
3,2005-07-31 12:06:41+00:00,2005-08-02 14:30:41+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401
4,2005-08-19 12:30:04+00:00,2005-08-23 13:35:04+00:00,2.99,2005.0,2.99,126.0,16.99,"{Trailers,""Behind the Scenes""}",0,0,0,1,8.9401,15876.0,8.9401


## Initial check

In [91]:
rental_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   rental_date       15861 non-null  datetime64[ns, UTC]
 1   return_date       15861 non-null  datetime64[ns, UTC]
 2   amount            15861 non-null  float64            
 3   release_year      15861 non-null  float64            
 4   rental_rate       15861 non-null  float64            
 5   length            15861 non-null  float64            
 6   replacement_cost  15861 non-null  float64            
 7   special_features  15861 non-null  object             
 8   NC-17             15861 non-null  int64              
 9   PG                15861 non-null  int64              
 10  PG-13             15861 non-null  int64              
 11  R                 15861 non-null  int64              
 12  amount_2          15861 non-null  float64            
 13  l

In [92]:
rental_info.describe()

,amount,release_year,rental_rate,length,replacement_cost,NC-17,PG,PG-13,R,amount_2,length_2,rental_rate_2
count,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000,15861.000000
mean,4.217161,2006.885379,2.944101,114.994578,20.224727,0.204842,0.200303,0.223378,0.198726,23.355504,14832.841876,11.389287
std,2.360383,2.025027,1.649766,40.114715,6.083784,0.403599,0.400239,0.416523,0.399054,23.503164,9393.431996,10.005293
min,0.990000,2004.000000,0.990000,46.000000,9.990000,0.000000,0.000000,0.000000,0.000000,0.980100,2116.000000,0.980100
25%,2.990000,2005.000000,0.990000,81.000000,14.990000,0.000000,0.000000,0.000000,0.000000,8.940100,6561.000000,0.980100
50%,3.990000,2007.000000,2.990000,114.000000,20.990000,0.000000,0.000000,0.000000,0.000000,15.920100,12996.000000,8.940100
75%,4.990000,2009.000000,4.990000,148.000000,25.990000,0.000000,0.000000,0.000000,0.000000,24.900100,21904.000000,24.900100
max,11.990000,2010.000000,4.990000,185.000000,29.990000,1.000000,1.000000,1.000000,1.000000,143.760100,34225.000000,24.900100


In [93]:
rental_info.isnull().sum()

rental_date         0
return_date         0
amount              0
release_year        0
rental_rate         0
length              0
replacement_cost    0
special_features    0
NC-17               0
PG                  0
PG-13               0
R                   0
amount_2            0
length_2            0
rental_rate_2       0
dtype: int64

No missing values. Next step, we will add the `rental_length_days` using the different between `return_date` and `rental_date` using the instruction :
- Create a column named `rental_length_days` using the columns `return_date` and `rental_date`, and add it to the pandas DataFrame. This column should contain information on how many days a DVD has been rented by a customer.

In [94]:
# Calculate the rental length as a Timedelta
rental_info['rental_length'] = rental_info['return_date'] - rental_info['rental_date']

# Convert the Timedelta to total number of days as a float
rental_info['rental_length_days'] = rental_info['rental_length'].dt.total_seconds() / (24 * 3600)

# Drop the temporary 'rental_length' column if you don't need it
rental_info = rental_info.drop(columns=['rental_length'])

# Check the result
print(rental_info[['rental_date', 'return_date', 'rental_length_days']].head())

                rental_date               return_date  rental_length_days
0 2005-05-25 02:54:33+00:00 2005-05-28 23:40:33+00:00            3.865278
1 2005-06-15 23:19:16+00:00 2005-06-18 19:24:16+00:00            2.836806
2 2005-07-10 04:27:45+00:00 2005-07-17 10:11:45+00:00            7.238889
3 2005-07-31 12:06:41+00:00 2005-08-02 14:30:41+00:00            2.100000
4 2005-08-19 12:30:04+00:00 2005-08-23 13:35:04+00:00            4.045139


Create two columns of dummy variables from "special_features", which takes the value of 1 when:
The value is "Deleted Scenes", storing as a column called "deleted_scenes".
The value is "Behind the Scenes", storing as a column called "behind_the_scenes". According to the instruction, we need to explore the details of the `special_features` columns.

In [95]:
# Check the unique values

rental_info['special_features'].unique()

array(['{Trailers,"Behind the Scenes"}', '{Trailers}',
       '{Commentaries,"Behind the Scenes"}', '{Trailers,Commentaries}',
       '{"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes","Behind the Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes"}',
       '{"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes","Behind the Scenes"}',
       '{Commentaries,"Deleted Scenes"}', '{Commentaries}',
       '{Trailers,Commentaries,"Behind the Scenes"}',
       '{Trailers,"Deleted Scenes"}', '{"Deleted Scenes"}',
       '{Trailers,Commentaries,"Deleted Scenes","Behind the Scenes"}'],
      dtype=object)

Crete the function to manipulate and clean the value.

In [96]:
def parse_special_features(s):
    # Remove braces
    s = s.strip('{}')
    # Split by comma
    features = s.split(',')
    # Clean up each feature by stripping whitespace and quotation marks
    features = [f.strip().strip('"').strip("'") for f in features]
    return features

In [97]:
# For 'deleted_scenes'
rental_info['deleted_scenes'] = rental_info['special_features'].apply(
    lambda x: int('Deleted Scenes' in parse_special_features(x))
)

# For 'behind_the_scenes'
rental_info['behind_the_scenes'] = rental_info['special_features'].apply(
    lambda x: int('Behind the Scenes' in parse_special_features(x))
)

In [98]:
# Verify the Results
rental_info[['special_features', 'deleted_scenes', 'behind_the_scenes']].head()

,special_features,deleted_scenes,behind_the_scenes
0,"{Trailers,""Behind the Scenes""}",0,1
1,"{Trailers,""Behind the Scenes""}",0,1
2,"{Trailers,""Behind the Scenes""}",0,1
3,"{Trailers,""Behind the Scenes""}",0,1
4,"{Trailers,""Behind the Scenes""}",0,1


In [99]:
# Drop the object feature
cols_to_drop = ["special_features", "rental_length_days", "rental_date", "return_date"]

In [100]:
# Recheck the data type before train the model
rental_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype              
---  ------              --------------  -----              
 0   rental_date         15861 non-null  datetime64[ns, UTC]
 1   return_date         15861 non-null  datetime64[ns, UTC]
 2   amount              15861 non-null  float64            
 3   release_year        15861 non-null  float64            
 4   rental_rate         15861 non-null  float64            
 5   length              15861 non-null  float64            
 6   replacement_cost    15861 non-null  float64            
 7   special_features    15861 non-null  object             
 8   NC-17               15861 non-null  int64              
 9   PG                  15861 non-null  int64              
 10  PG-13               15861 non-null  int64              
 11  R                   15861 non-null  int64              
 12  amount_2            15861 non-nu

- Make a pandas DataFrame called X containing all the appropriate features you can use to run the regression models, avoiding columns that leak data about the target.
- Choose the "rental_length_days" as the target column and save it as a pandas Series called y.

In [102]:
X = rental_info.drop(columns=cols_to_drop)
y = rental_info['rental_length_days']

- Split the data into X_train, y_train, X_test, and y_test train and test sets, avoiding any features that leak data about the target variable, and include 20% of the total data in the test set.
- Set random_state to 9 whenever you use a function/method involving randomness, for example, when doing a test-train split.

In [103]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=9)

train_data = [X_train, y_train, X_test, y_test]
data_names = ['X_train', 'y_train', 'X_test', 'y_test']

for name, data in zip(data_names, train_data):
    print(f'Data Shape of {name}: {data.shape}')

Data Shape of X_train: (12688, 14)
Data Shape of y_train: (12688,)
Data Shape of X_test: (3173, 14)
Data Shape of y_test: (3173,)


## Model Training

In [104]:
# Create the parameter grid
param_grid = {
    'n_estimators' : [50, 100],
    'max_depth' : [3, 5],
    'learning_rate' : [0.1, 0.01],
    'colsample_bytree' : [0.8, 1.0]
}

# Instantiate the regression model
xgb_re = xgb.XGBRegressor(random_state=9)

# GridSearchCV
grid_mse = GridSearchCV(param_grid=param_grid, 
                        estimator=xgb_re,
                        scoring='neg_mean_squared_error',
                        cv = 5,
                        verbose=1,
                        n_jobs=1
                       )

# Fit the model
grid_mse.fit(X_train, y_train)

# Print the best score
print(f"Best Score Parameter : {grid_mse.best_params_}")
print(f"Best Negative MSE Score: {grid_mse.best_score_}")

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Score Parameter : {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 5, 'n_estimators': 100}
Best Negative MSE Score: -1.9440109714236287


In [105]:
# Predict on the test set
y_pred = grid_mse.predict(X_test)

best_model = xgb_re
# Calculate MSE on the test set
best_mse = mean_squared_error(y_test, y_pred)
print(f"Test MSE: {best_mse}")

Test MSE: 2.0051738322721353
